# **With this file you will configure the enviroment to start training the object detector**

# **1. Check that the GPU can be recognized**

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
 
with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  #net_gpu = tf.keras.layers.Conv2D(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)
   
sess = tf.Session(config=config)
 
try:
  sess.run(tf.global_variables_initializer())
  print('ok')
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

# **2. Setting up the virtual environment for tensorflow-gpu**

In [0]:
#install the nessesary libraries
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

In [0]:
!pip install -q Cython contextlib2 pillow lxml matplotlib pandas opencv-python

In [0]:
!pip install -q pycocotools

# **3. Mount google drive and navigate to the workshop folder**

In [0]:
from os.path import join
from google.colab import drive
import os
 
ROOT = "/content/drive"
drive.mount(ROOT)

# This is a custom path. 'My Drive' is the root of your google drive change accordingly
PROJ = "My Drive/workshop" 
PROJECT_PATH = join(ROOT, PROJ)
 
#%cd ~/content 
%cd {PROJECT_PATH}

print ("The current directory is:"+os.getcwd())

# **4. Very important add the following folders to the pythopath**

Tensorflow Object detection API needs to call some files from the structure of the "models" folder, thats why we add it to the pythopath

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/workshop/models/:/content/drive/My Drive/workshop/models/research/:/content/drive/My Drive/workshop/models/research/slim/'
print(  os.environ['PYTHONPATH'])


**Move to the research folder and compile the protoc files (this has to be done only once)**

In [0]:
#move to the research folder first 
%cd /content/drive/My Drive/workshop/models/research


In [0]:

#compiling the protoc files
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
#build the setup file
!python setup.py build

In [0]:
#install the setup file
!python setup.py install

# **5. Run tests**
We need to check if everything is running

In [0]:
#The current directory should be:/content/drive/My Drive/workshop/models/research

import os
cwd = os.getcwd()
print ("The current directory is:"+cwd)

#Change the folder if not
#!cd '/content/drive/My Drive/workshop/models/research'

In [0]:

!python object_detection/builders/model_builder_test.py

#if the tests fail check the pythopath
# ModuleNotFoundError: No module named 'nets' => check pythopath

#if ModuleNotFoundError: No module named 'object_detection' =>  run setup and build again

# **6. Prepare the training configuration**

**Create CSV files from the xml anotations**

In [0]:
#make sure you are in object detection folder

print(os.getcwd())

#if not change
#%cd object_detection/

In [0]:
!python xml_to_csv.py

 **Generate TFrecord Files**

modify the "generate_tfrecord.py" file with the corresponding class labels in line 30


In [0]:
#generate tfrecord for training data
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

In [0]:
#generate tfrecord for test data
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

 **Configure faster_rcnn_inception_v2_pets.config file in the training folder**


1.   Line 9. Change num_classes to the number of different objects you want the classifier to detect. For the above example, it would be num_classes : 6 .

2.   Line 106. Change fine_tune_checkpoint to:
        fine_tune_checkpoint : "./faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt

3.   Line 113. Change num_steps to:
num_steps: 10000

4.   Lines 123 and 125. In the train_input_reader section, change input_path and label_map_path to:
        123 - input_path : "./train.record"
        125 - label_map_path: "./training/labelmap.pbtxt"

5.   Line 130. Change num_examples to the number of images you have in the \images\test directory.

6.   Lines 135 and 137. In the eval_input_reader section, change input_path and label_map_path to:
        input_path : "./test.record" 
        label_map_path: "./training/labelmap.pbtxt"




**Create and configure labelmap.pbtxt in workshop/models/research/training/**

In [0]:
with open('training/labelmap.pbtxt', 'w') as f:
  f.write("item {"+'\n'+
            '\t'+"id: 1"+'\n'+
            '\t'+"name: 'nine'"+'\n'+
          "}"+'\n'+

          "item {"+'\n'+
            '\t'+"id: 2"+'\n'+
            '\t'+"name: 'ten'"+'\n'+
          "}"+'\n'+

          "item {"+'\n'+
            '\t'+"id: 3"+'\n'+
            '\t'+"name: 'jack'"+'\n'+
          "}"+'\n'+

          "item {"+'\n'+
            '\t'+"id: 4"+'\n'+
            '\t'+"name: 'queen'"+'\n'+
          "}"+'\n'+

          "item {"+'\n'+
            '\t'+"id: 5"+'\n'+
            '\t'+"name: 'king'"+'\n'+
          "}"+'\n'+

          "item {"+'\n'+
            '\t'+"id: 6"+'\n'+
            '\t'+"name: 'ace'"+'\n'+
          "}")

In [0]:
!cat 'training/labelmap.pbtxt'



# **Move train.py from object_detection/legacy folder one up to /object_detection**

# **7. Now we train!!!!**

**Prepare a Tensorboard** (optional)



Tensorboard is a GUI to show the advance of our trainig 

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
model_dir = 'training/'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

Get tensorboard link

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

**Make sure you are in the object detection folder**

In [0]:
print(os.getcwd())

In [0]:
#if not, move accordingly
#%cd object_detection/

In [0]:
#run the training and keep this window open until getting the desired loss (ex 0.01)
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

In [0]:
#once the trainig is done create a frozen_inference_graph file to do the predictions in the notebook
#change the checkpoint to the last one registered
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-3312 --output_directory inference_graph